In [56]:
import mne
from mne.datasets import eegbci
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.multiclass import OneVsRestClassifier
from mne.decoding import CSP

In [57]:
subject_id = 1
runs_lr = [4, 8, 12] # right or left hand fists data
runs_feet = [6, 10, 14] # feet or both fists data

In [58]:
print("Downloading data...")
fnames_lr = eegbci.load_data(subject_id, runs = runs_lr, update_path = True)
print("Download complete.")

Download complete.


In [59]:
print("Downloading data...")
fnames_feet = eegbci.load_data(subject_id, runs = runs_feet, update_path = True)
print("Download complete.")

Download complete.


In [60]:
raws_lr = [mne.io.read_raw_edf(f, preload = True) for f in fnames_lr]

Extracting EDF parameters from /Users/Mohammad/mne_data/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from /Users/Mohammad/mne_data/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R08.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from /Users/Mohammad/mne_data/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R12.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...


In [61]:
raws_feet = [mne.io.read_raw_edf(f, preload = True) for f in fnames_feet]

Extracting EDF parameters from /Users/Mohammad/mne_data/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R06.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from /Users/Mohammad/mne_data/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R10.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from /Users/Mohammad/mne_data/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R14.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...


In [62]:
raw_lr = mne.concatenate_raws(raws_lr)
raw_feet = mne.concatenate_raws(raws_feet)

In [63]:
def process_and_epoch(raw, event_id):
    raw.filter(l_freq=8., h_freq=35.)
    raw.notch_filter(freqs=50)
    events, _ = mne.events_from_annotations(raw, event_id={'T1': 1, 'T2': 2})
    epochs = mne.Epochs(raw, events, event_id, tmin=-1., tmax=4., preload=True, baseline=None, picks='eeg')
    return epochs

In [64]:
epochs_lr = process_and_epoch(raw_lr, event_id={'left_fist': 1, 'right_fist': 2})
epochs_feet = process_and_epoch(raw_feet, event_id={'feet': 2})

Filtering raw data in 3 contiguous segments
Setting up band-pass filter from 8 - 35 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 35.00 Hz
- Upper transition bandwidth: 8.75 Hz (-6 dB cutoff frequency: 39.38 Hz)
- Filter length: 265 samples (1.656 s)

Filtering raw data in 3 contiguous segments
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edg

In [65]:
epochs = mne.concatenate_epochs([epochs_lr, epochs_feet])

Not setting metadata
69 matching events found
No baseline correction applied


/var/folders/1s/bkqkh8453p79rr15mcq_bm4c0000gp/T/ipykernel_52314/3562426782.py:1: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  epochs = mne.concatenate_epochs([epochs_lr, epochs_feet])


In [66]:
labels = epochs.events[:, -1]
data = epochs.get_data()

In [67]:
csp = CSP(n_components=4, reg=None, log=True)
lda = LDA()
ovr_pipeline = OneVsRestClassifier(Pipeline([('CSP', csp), ('LDA', lda)]))

In [68]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(ovr_pipeline, data, labels, cv=cv)

Computing rank from data with rank=None
    Using tolerance 0.00037 (2.2e-16 eps * 64 dim * 2.6e+10  max singular value)
    Estimated rank (data): 64
    data: rank 64 computed from 64 data channels with 0 projectors
Reducing data rank from 64 -> 64
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 0.00038 (2.2e-16 eps * 64 dim * 2.7e+10  max singular value)
    Estimated rank (data): 64
    data: rank 64 computed from 64 data channels with 0 projectors
Reducing data rank from 64 -> 64
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 0.00038 (2.2e-16 eps * 64 dim * 2.7e+10  max singular value)
    Estimated rank (data): 64
    data: rank 64 computed from 64 data channels with 0 projectors
Reducing data rank from 64 -> 64
Estimating class=0 covariance using EMP

In [69]:
print("\n--- 3-Class Cross-Validation Accuracy ---")
print(f"Scores for each fold: {scores}")
print(f"Mean Accuracy: {np.mean(scores):.4f}")
print(f"Standard Deviation: {np.std(scores):.4f}")


--- 3-Class Cross-Validation Accuracy ---
Scores for each fold: [0.57142857 0.35714286 0.64285714 0.71428571 0.92307692]
Mean Accuracy: 0.6418
Standard Deviation: 0.1846
